## acs_etl.load

Create table and load data into the specified database.

### Source Code

In [ ]:
# default_exp load

In [ ]:
#export
import os
os.chdir('../')
os.getcwd()

'E:\\OneDriveLocal\\OneDrive\\学习\\Graduate Study\\2021Fall\\mlpp\\etl_demo'

In [ ]:
#export
from nbdev.showdoc import *
from acs_etl.acs import ACS_Blockgroup_Data_Pull
import numpy as np
import pandas as pd
import psycopg2
import re

In [ ]:
# export
def database_connector(func):
    def with_connection(*args, **kwargs):

        conn = psycopg2.connect(
            host='acs-db.mlpolicylab.dssg.io',
            database='acs_data_loading',
            user='mlpp_student',
            password='CARE-horse-most')
        cur = conn.cursor()

        try:
            func(*args, cur, **kwargs)
            conn.commit()

        except Exception as e:
            print(e)

        finally:
            cur.close()
            conn.close()

    return with_connection

In [ ]:
#export
def series_to_tuple_like_string(series):
    '''Transform a pandas series into a tuple-like string to fit in SQL queries.
    '''
    string = '('
    for item in series:
        if not re.match(r'^[0-9]+$', str(item)):
            string = string + "'" + str(item) + "'" + ','
        else:
            string = string + str(item) + ','
    string = string.strip(',') + ')'
    return string

In [ ]:
#export
class ACS_Blockgroup_Data_Loader:
    
    def __init__(self, df, db_dict, schema, table):
        self.df: pd.DataFrame = df
        self.db_dict: dict = db_dict
        self.schema: str = schema
        self.table: str = table
            
        self.host = db_dict['host']
        self.database = db_dict['database']
        self.user = db_dict['user']
        self.password = db_dict['password']
    
    @database_connector
    def create_table(self, cur):
        '''Create a table with the specified name.
        '''
        cur.execute(f"CREATE TABLE acs.{self.table} (\
                     state_id INTEGER,\
                     state VARCHAR(4),\
                     county_id INTEGER,\
                     county VARCHAR(40),\
                     blockgroup INTEGER,\
                     tract INTEGER,\
                     total_population NUMERIC,\
                     median_age NUMERIC,\
                     per_capita_income NUMERIC,\
                     median_rent_to_household_income NUMERIC,\
                     avg_household_size NUMERIC,\
                     NAME VARCHAR(100)\
                   )")
    
    @database_connector
    def insert_data(self, cur):
        '''Insert rows in df into the specified table.
        '''
        for i, row in self.df.iterrows():
            row_string = series_to_tuple_like_string(row)
            cur.execute(f"INSERT INTO acs.{self.table} VALUES {row_string}")
            

### Executing Loading Process

<b><font color='#C12340'>CAUTION: THIS SECTION COULD AFFECT THE SHARED DATABASE, THINK TWICE BEFORE EXECUTING.</font></b>

In [ ]:
db_dict = {'host': 'acs-db.mlpolicylab.dssg.io',
           'database': 'acs_data_loading',
           'user': 'mlpp_student', 
           'password': 'CARE-horse-most'}
df = ACS_Blockgroup_Data_Pull().cleaned_data
loader = ACS_Blockgroup_Data_Loader(df, db_dict, 'acs', 'chengyu2_acs_data')
loader.create_table()
loader.insert_data()

### Simple Test Example

In [ ]:
conn = psycopg2.connect(
         host='localhost',
         database='postgres',
         user='postgres',
         password='1622')

cur = conn.cursor()

cur.execute("INSERT INTO acs.chengyu2_acs_data2 VALUES (0, 24, 12)")

conn.commit()

cur.close()

conn.close()